In [ ]:
# -*- coding: utf-8 -*-
import os
import gc
import time
import heapq
import json
import itertools
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

INF = 1e12  # Represents infinity for cost
os.system('cls' if os.name == 'nt' else 'clear')

# Input/output directories
input_folder = r'C:\Users\Murilo\Documents\DOUTORADO\UFMS-FACOM\Material-Ricardo\Encontros-Ricardo\Encontro_55-08-07-2025\25nds\input'
output_folder = r'C:\Users\Murilo\Documents\DOUTORADO\UFMS-FACOM\Material-Ricardo\Encontros-Ricardo\Encontro_55-08-07-2025\25nds\output'
os.makedirs(output_folder, exist_ok=True)
PARQUET_COST_DIR = os.path.join(output_folder, 'cost_matrix_parquet')
DATA_DIR = os.path.join(output_folder, 'parquet_r2_jobs')
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(PARQUET_COST_DIR, exist_ok=True)

# Compute shortest path latency from source using Dijkstra
def get_latencies(source, adjList, numNodes):
    latencies = [INF] * numNodes
    latencies[source] = 0
    heap = [(0, source)]
    while heap:
        curr_lat, u = heapq.heappop(heap)
        if curr_lat > latencies[u]:
            continue
        for v, weight in adjList[u]:
            if latencies[v] > curr_lat + weight:
                latencies[v] = curr_lat + weight
                heapq.heappush(heap, (latencies[v], v))
    return latencies

# Save valid combinations to a Parquet file
def save_comb_to_parquet(job_id, comb_list):
    if not comb_list:
        df = pd.DataFrame(columns=["node1"])
    else:
        max_len = max(len(c) for c in comb_list)
        col_names = [f'node{i+1}' for i in range(max_len)]
        df = pd.DataFrame(comb_list, columns=col_names)
    table = pa.Table.from_pandas(df)
    pq.write_table(table, os.path.join(DATA_DIR, f'job_{job_id:05d}.parquet'), compression='snappy')

# Generate all valid node combinations of size 1 and 2 and save them
def generate_combinations_r1_r2_and_save(data):
    jr, jb, jl, jo = map(np.array, (data['jr'], data['jb'], data['jl'], data['jo']))  # Job requirements
    V_R, V_B = map(np.array, (data['V_R'], data['V_B']))  # Node capabilities
    adjList = data['adjList']
    num_nodes = len(V_R)
    num_jobs = len(jr)
    print(f"\nRunning Scenario: {num_nodes} nds - {num_jobs} jobs. Waiting...\n")

    for job_id in tqdm(range(num_jobs), desc="Generating and saving combinations"):
        comb_validas = []
        jrj, jbj, jlj, joj = jr[job_id], jb[job_id], jl[job_id], jo[job_id]
        N_Latency = get_latencies(joj, adjList, num_nodes)
        for r in [1, 2]:  # Only 1-node and 2-node combinations
            for comb in itertools.combinations(range(num_nodes), r):
                total_R = sum(V_R[n] for n in comb)
                total_B = sum(V_B[n] for n in comb)
                total_L = sum(N_Latency[n] for n in comb)
                if (total_R >= jrj and total_B >= jbj and total_L <= jlj - 1):
                    comb_validas.append(tuple(comb))
        save_comb_to_parquet(job_id, comb_validas)
        gc.collect()
    return num_nodes, num_jobs

# Lazily load combinations from Parquet to reduce memory usage
def load_parquet_combinations_lazy(job_id):
    path = os.path.join(DATA_DIR, f'job_{job_id:05d}.parquet')
    pf = pq.ParquetFile(path)
    for batch in pf.iter_batches():
        df = batch.to_pandas()
        for row in df.itertuples(index=False):
            yield tuple(int(v) for v in row if not pd.isna(v))

# Save job cost matrix to Parquet
def save_cost_matrix_to_parquet(job_id, costs_row, combs):
    df = pd.DataFrame({'comb': list(map(str, combs)), 'cost': costs_row})
    table = pa.Table.from_pandas(df)
    pq.write_table(table, os.path.join(PARQUET_COST_DIR, f'job_{job_id:05d}.parquet'), compression='snappy')

# Compute and store cost matrix for all jobs
def calculate_cost_matrix_streaming(data):
    jr, jb, jl, jo = map(np.array, (data['jr'], data['jb'], data['jl'], data['jo']))
    V_R, V_B = map(np.array, (data['V_R'], data['V_B']))
    adjList = data['adjList']
    num_jobs = len(jr)
    num_nodes = len(V_R)
    unique_combs_set = set()
    for job_id in range(num_jobs):
        for comb in load_parquet_combinations_lazy(job_id):
            unique_combs_set.add(tuple(sorted(comb)))
    unique_combs = list(unique_combs_set)
    latency_cache = {o: get_latencies(o, adjList, num_nodes) for o in np.unique(jo)}
    for job_id in tqdm(range(num_jobs), desc="Saving cost matrix in parquet"):
        origin = jo[job_id]
        l_job = jl[job_id] - 1
        N_Latency = latency_cache[origin]
        row_costs = []
        for comb in unique_combs:
            total_R = sum(V_R[n] for n in comb)
            total_B = sum(V_B[n] for n in comb)
            total_L = sum(N_Latency[n] for n in comb)
            if total_R < jr[job_id] or total_B < jb[job_id] or total_L > l_job:
                row_costs.append(INF)
            else:
                cost = (total_R - jr[job_id]) ** 2 + (total_B - jb[job_id]) ** 2 - (l_job - total_L)
                row_costs.append(cost)
        save_cost_matrix_to_parquet(job_id, row_costs, unique_combs)
        gc.collect()
    return unique_combs, jr, jb, jl, jo

# Load cost vector for a job
def load_job_costs_parquet(job_id):
    df = pd.read_parquet(os.path.join(PARQUET_COST_DIR, f'job_{job_id:05d}.parquet'))
    return df['cost'].values.tolist()

# Estimate lower bound of cost for remaining jobs (used in BnB)
def calculate_lower_bound_parquet(start_job, assigned_nodes, combs, n_jobs):
    lb = 0
    for j in range(start_job, n_jobs):
        costs = load_job_costs_parquet(j)
        filtered = [c for idx, c in enumerate(costs) if not set(combs[idx]).intersection(assigned_nodes)]
        if filtered:
            lb += min(filtered)
        else:
            return INF
    return lb

# Branch and Bound optimization algorithm
def branch_and_bound_parquet(combs, n_jobs):
    best_cost = INF
    best_solution = None
    heap = [(0, [], 0, set())]  # (lower bound, partial sol, cost, assigned nodes)
    while heap:
        lb, partial, curr_cost, assigned = heapq.heappop(heap)
        job = len(partial)
        if job == n_jobs:
            if curr_cost < best_cost:
                best_cost = curr_cost
                best_solution = partial
            continue
        job_costs = load_job_costs_parquet(job)
        for idx, comb in enumerate(combs):
            if set(comb).intersection(assigned):
                continue
            new_cost = curr_cost + job_costs[idx]
            new_assigned = assigned.union(comb)
            est_lb = new_cost + calculate_lower_bound_parquet(job + 1, new_assigned, combs, n_jobs)
            if est_lb >= best_cost:
                continue
            heapq.heappush(heap, (est_lb, partial + [(job, idx)], new_cost, new_assigned))
    if best_solution is None:
        return None, None
    return [(j, combs[idx]) for j, idx in best_solution], [load_job_costs_parquet(j)[idx] for j, idx in best_solution]

# Print and save results to output file
def print_summary(solution, costs, jr, jb, jl, jo, input_path, output_folder, runtime, num_nodes, num_jobs):
    input_file = os.path.basename(input_path).split('.')[0]
    out_path = os.path.join(output_folder, f"results_parquet_{input_file}.txt")
    with open(out_path, 'w') as f:
        f.write("-" * 62 + "\n")
        f.write("-" * 22 + " Branch and Bound " + "-" * 22 + "\n")
        f.write("-" * 62 + "\n")
        f.write(f" Scenario: {num_nodes} nds - {num_jobs} jobs\n")
        f.write("-" * 62 + "\n")
        f.write(f"Input file: {input_file}\n\n")
        f.write(" Job   [ Jr,  Jb,  Jl,  Jo]".ljust(50) + "OF".rjust(20) + "Allocated Nodes".rjust(25) + "\n")

        total_cost = 0.0
        for i, (job_idx, nodes) in enumerate(solution):
            cost = costs[i]
            total_cost += cost
            f.write(f"  {job_idx:<3}  [{int(jr[job_idx])}, {int(jb[job_idx])}, {int(jl[job_idx])}, {int(jo[job_idx])}]".ljust(50))
            f.write(f"{cost:>20,.1f}".rjust(20) + f"{str(list(nodes)).rjust(25)}\n")

        f.write(f"\nTotal OF: {total_cost:,.1f}\n")
        f.write(f"\nRuntime: {runtime:,.4f} seconds\n")

    print(f"\n📁 Results saved in: {out_path}")

# Main execution logic
def main():
    for json_file in [f for f in os.listdir(input_folder) if f.endswith('.json')]:
        input_path = os.path.join(input_folder, json_file)
        with open(input_path, 'r') as f:
            data = json.load(f)
        start_total = time.time()
        num_nodes, num_jobs = generate_combinations_r1_r2_and_save(data)
        combs_all, jr, jb, jl, jo = calculate_cost_matrix_streaming(data)
        solution, costs = branch_and_bound_parquet(combs_all, num_jobs)
        end_total = time.time()
        print_summary(solution, costs, jr, jb, jl, jo, input_path, output_folder, end_total - start_total, num_nodes, num_jobs)
        gc.collect()

if __name__ == '__main__':
    main()
